In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list): # = "sorted" function
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data


def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data


def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def constantTF(listo):
    first=listo[0]
    ln=len(listo)
    for j in [0..ln-2]:
        if listo[j]!=listo[j+1]:return False
    return True

def push(listo):
    pushed=[]
    lenth=len(listo)
    pushed=pushed+[listo[lenth-1]]
    for j in [0..lenth-2]:
        pushed=pushed+[listo[j]]
    return pushed

def kIteratesofPush(listo,k):
    if k == 0:return listo
    answer=listo
    for k in [1..k]:
        answer=push(answer)
    return answer

def cycleEquivalentTF(list1,list2):
    len1=len(list1)
    len2=len(list2)
    if len1 != len2:return False
    if list1==list2:return True
    test=list2
    for k in [1..len2]:
        test=push(test)
        if test==list1:return True
    return False

def fromDigitsToNumber(listo,prime):
    ln=len(listo)
    sum=0
    for j in [0..ln-1]:
        digit=listo[j]
        sum=sum+digit*prime^j
    return sum

def numberListToExpansionList(seq,prime):
    el=[]
    ln=len(seq)
    for j in [0..ln-1]:
        nbr=seq[j]
        dgt=nbr.digits(prime)
        el=el+[dgt]
    return el

def expansionListToNumberList(listlist,prime):
    nlist=[]
    ln=len(listlist)
    if ln==0:return []
    for j in [0..ln-1]:
        lst=listlist[j]
        nbr=fromDigitsToNumber(lst,prime)
        nlist=nlist+[nbr]
    return nlist
        
def f5(seq, idfun=None): # https://www.peterbe.com/plog/uniqifiers-benchmark
    if idfun is None:
        def idfun(x): return x
        seen = {}
        result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

def drop_duplicates(seq):return f5(seq,idfun=None) # for numerical sequences only

def dropCycle_EquivalentLists(listlist): # drops duplicates
    if listlist==[]:return []
    ddl=[listlist[0]]
    for lst in listlist:
        flag = 1
        for listo in ddl:
            if cycleEquivalentTF(lst,listo):
                flag=0
        if flag==1:ddl=ddl+[lst]
    return ddl  

def zeroLogsOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def zeroLogsOverFieldDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            di=i.digits(prime)
            answer=answer+[di]
    return answer

def last(listo):
    if listo==[]:return None
    if len(listo)>0:return listo[len(listo)-1]

def cycleEquivalenceClass(listo):
    if listo==[]:return []
    classs=[listo]
    while len(classs)<len(listo):
        classs=classs+[push(last(classs))]
    return classs
    

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [2]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
no=[]
bound=120
for k in [0..len(s)-1]:
    print k
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    lm=logsAndMultiplicitiesOverField(poly,prime,sd)
    logs=[]
    digits=[]
    for j in [0..len(lm)-1]:
        log=lm[j][0]
        logs=logs+[log]
    logs=sorted(logs)
    count=0
    for j in [0..len(lm)-1]:
        log=logs[j]
        digit=log.digits(prime)
        digit=padListOnRightToLengthK(digit,sd)
        tf=constantTF(digit)
        if tf==False:count=count+1
    if count!=sd&sd>1:no=no+[index] #no clear regularities
print no

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
[8, 11, 17, 23, 34, 38, 49, 50, 80, 83, 85, 104, 106, 116, 125, 136, 142, 160, 169, 178, 187, 188, 194, 196]


In [3]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=0
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(0, 1)
logs:
[]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(3, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(6, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(9, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(12, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
----------------

(117, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(120, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(123, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(126, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(129, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]

In [4]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=1
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(1, 1)
logs:
[-1]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(4, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(7, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(10, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(13, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
-------------

(130, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(133, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(136, 2)
logs:
[-1, 1, 2, 3, 4, 5, 6, 7, 8]
cycle-equivalence class representatives as expansions
[[-1], [1], [2], [2, 1]]
cycle-equivalence class representatives as numbers
[-1, 1, 2, 5]
------------------------------------------------------------------------------
(139, 1)
logs:
[-1, 1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(142, 2)
logs:
[-1, 1, 2, 3, 4, 5, 6, 7, 8]
cycle-equivalence class representatives a

In [5]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=2
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(-1, 1)
logs:
[]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(2, 1)
logs:
[1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(5, 1)
logs:
[1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(8, 3)
logs:
[4, 10, 12, 13, 17, 23, 25, 26]
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[4, 17]
------------------------------------------------------------------------------
(11, 3)
logs:
[4, 10, 12, 13, 17, 23, 25, 26]
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 

(113, 1)
logs:
[1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(116, 2)
logs:
[1, 2, 3, 4, 5, 6, 7, 8]
cycle-equivalence class representatives as expansions
[[1], [2], [2, 1]]
cycle-equivalence class representatives as numbers
[1, 2, 5]
------------------------------------------------------------------------------
(119, 1)
logs:
[1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(122, 1)
logs:
[1, 2]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(125, 2)
logs:
[1, 2, 3, 4, 5, 6, 7, 8]
cycle-equivalence class representatives as expansions
[[1], [2], [2, 1]

In [10]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=2
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        if sd==3:
            rd=ord(3,index-residue)
            print (index,sd,rd)
            print "cycle-equivalence class representatives as expansions"
            print expansions
            print "cycle-equivalence class representatives as numbers"
            print numbers
            print "------------------------------------------------------------------------------"

(8, 3, 1)
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[4, 17]
------------------------------------------------------------------------------
(11, 3, 2)
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[4, 17]
------------------------------------------------------------------------------
(23, 3, 1)
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[4, 17]
------------------------------------------------------------------------------
(38, 3, 2)
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[4, 17]
------------------------------------------------------------------------------
(50, 3, 1)
cycle-equivalence class representatives as expansions
[[1, 1], [2, 2, 1]]
cycle-equivalence class 

In [11]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=1
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        if sd==3:
            rd=ord(3,index-residue)
            print (index,sd,rd)
            print "cycle-equivalence class representatives as expansions"
            print expansions
            print "cycle-equivalence class representatives as numbers"
            print numbers
            print "------------------------------------------------------------------------------"

(49, 3, 1)
cycle-equivalence class representatives as expansions
[[-1], [1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[-1, 4, 17]
------------------------------------------------------------------------------
(106, 3, 1)
cycle-equivalence class representatives as expansions
[[-1], [1], [2], [2, 1], [1, 2], [2, 2], [2, 1, 1]]
cycle-equivalence class representatives as numbers
[-1, 1, 2, 5, 7, 8, 14]
------------------------------------------------------------------------------
(169, 3, 1)
cycle-equivalence class representatives as expansions
[[-1], [1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[-1, 4, 17]
------------------------------------------------------------------------------
(196, 3, 1)
cycle-equivalence class representatives as expansions
[[-1], [1, 1], [2, 2, 1]]
cycle-equivalence class representatives as numbers
[-1, 4, 17]
------------------------------------------------------------------------------


In [13]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
residue=0
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        if sd==3:
            rd=ord(3,index-residue)
            print (index,sd,rd)
            print "cycle-equivalence class representatives as expansions"
            print expansions
            print "cycle-equivalence class representatives as numbers"
            print numbers
            print "------------------------------------------------------------------------------"